## BCC结构W中空位形成能的计算，与空位形成能对应的是自间隙子形成能

In [ ]:
%%writefile day3.in
#初始化系统
units metal
atom_style atomic
boundary  p p p
neighbor 0.3 bin
# 构建钨模型
lattice bcc 3.168
region box block 0 10 0 10 0 10
create_box 1 box
create_atoms 1 box
# 定义相互作用势
pair_style eam/alloy
pair_coeff * * W_zhou.eam.alloy W
#定义热力学信息输出
thermo 10
dump 1 all custom 1 day3.xyz id type x y z
#结构优化
minimize 1e-12 1e-12 10000 10000
#记录优化后的能量与原子数目
variable E0 equal etotal
variable E_prefect equal ${E0}
variable N equal count(all)
variable N0 equal ${N}
#删除中心原子
region 2 sphere 5.5 5.5 5.5 0.1 side in
delete_atoms region 2
#在特定位置添加w原子
create_atoms 1 single 5.25 5.25 5.25
create_atoms 1 single 5.75 5.75 5.75
#结构优化得到E_defect能量
minimize 1e-12 1e-12 10000 10000
#记录二次优化能量值
variable E_defect equal etotal
variable Ev equal (${E_defect}-((${N0}+1)/${N0})*${E_prefect})
print "all done"
print "SIA formation energy = ${Ev} ev"

In [ ]:
!"E:/LAMMPS/bin/lmp" < day3.in

## Lammps 循环计算方法
`variable`  
`clear`  
`jump`  
`next`  
`quit`  
`variable i loop 5` 定义循环5次  
`variable seed index 666 888 999 1010 555`定义随机数seed   
`clear` 清除之前所有定义  
`next i`  
`next seed`将i和seed值换为下一个  
`jump relax.in` 进入`relax.in`文件重新执行其中的命令  
`quit` 跳出循环  
定义的变量i可以用于输出的文件名，来观察不同seed情况下的弛豫情况`dump 1 all custom 100 W.${i}.xyz  id  type x y z`



In [ ]:
%%writefile loop.in
#定义循环变量
variable i loop 5
variable seed index 666 888 999 1010 555
# 初始系统设置
units metal
atom_style atomic
boundary p p p
#初始模型构建
lattice bcc 3.168
region box block 0 5 0 5 0 5
create_box 1 box 
create_atoms 1 box 
#定义原子相互作用势
pair_style eam/alloy
pair_coeff * * W_zhou.eam.alloy W
#计算原子势能
compute 1 all pe/atom
#定义输出原子/体系信息
thermo 100
thermo_style custom step temp pe etotal dt time
dump 1 all custom 100 W.${i}.xyz id type x y z c_1
#设定运行环境并运行
velocity all create 300 ${seed} mom yes rot yes dist gaussian
fix 1 all nvt temp 300 300 0.05
timestep 0.001
run 1000
print "*******************************************"
print "JOB ${i} has been done"
print "*******************************************"
#进行下一次循环
clear
next i
next seed
jump loop.in
#循环结束
quit

In [ ]:
!"E:/LAMMPS/bin/lmp" < loop.in

## Fe中的辐照损伤模拟
用一个小体系为例简单进行模拟过程（这也是做MD的原则之一，不要一开始就跑大体系，先跑个小体系试试情况，没问题再上大体系）    
`unfix 1` 解除nvt系综(必须)  
`velocity pka set 300 500 600 units box` 给pka原子速度赋值  
`fix 2 all dt/reset 1 0.00000001 0.0005 0.005 units lattice`
由于碰撞过程中原子的速度过快，为了保证力和能量的计算精度，需要根据原子运行的距离控制时间步长，一般来说保证一个步长时间内，原子运行距离不能超过0.005a（a为晶格常数）

### fix
--- 每多少步基于原子速度与受力重置时间步长，防止原子移动距离过大   
`fix ID group-ID dt/reset N Tmin Tmax Xmax keyword values ...`  
- ID, group-ID are documented in fix command
- dt/reset = style name of this fix command
- N = re-compute dt every N timesteps
- Tmin = minimum dt allowed which can be NULL (time units)
- Tmax = maximum dt allowed which can be NULL (time units)
- Xmax = maximum distance for an atom to move in one timestep (distance units)
- zero or more keyword/value pairs may be appended
- keyword = emax or units  

```bash
emax value = Emax
  Emax = maximum kinetic energy change for an atom in one timestep (energy units)
units value = lattice or box
  lattice = Xmax is defined in lattice units
  box = Xmax is defined in simulation box units
```

### velocity
--- 设置原子速度  
`velocity group-ID style args keyword value`  
- group-ID = ID of group of atoms whose velocity will be changed  
- style = create or set or scale or ramp or zero
```bash
create args = temp seed
  temp = temperature value (temperature units)
  seed = random # seed (positive integer)
set args = vx vy vz
  vx,vy,vz = velocity value or NULL (velocity units)
  any of vx,vy,vz van be a variable (see below)
scale arg = temp
  temp = temperature value (temperature units)
ramp args = vdim vlo vhi dim clo chi
  vdim = vx or vy or vz
  vlo,vhi = lower and upper velocity value (velocity units)
  dim = x or y or z
  clo,chi = lower and upper coordinate bound (distance units)
zero arg = linear or angular
  linear = zero the linear momentum
  angular = zero the angular momentum
```
- zero or more keyword/value pairs may be appended
- keyword = dist or sum or mom or rot or temp or bias or loop or units  
```bash
dist value = uniform or gaussian
sum value = no or yes
mom value = no or yes
rot value = no or yes
temp value = temperature compute ID
bias value = no or yes
loop value = all or local or geom
rigid value = fix-ID
  fix-ID = ID of rigid body fix
units value = box or lattice
```

In [ ]:
%%writefile Fe.in
units metal
atom_style atomic
boundary p p p
lattice bcc 2.8552
region box block 0 30 0 30 0 30 units lattice  
create_box 1 box
create_atoms 1 box
region rpka sphere 15 15 15 0.2 units lattice
group pka region rpka
pair_style eam/fs
pair_coeff * * Fe_mm.eam.fs Fe
thermo 2000
thermo_style custom step temp etotal time
dump 1 all custom 500 Fe.xyz id type x y z
# nvt
velocity all create 1000 666 rot yes dist gaussian
fix 1 all nvt temp 300 300 0.05
timestep 0.001
run 15000
# nve
unfix 1 
velocity pka set 300 500 600 units box
fix 1 all nve 
fix 2 all dt/reset 1 0.00000001 0.0005 0.005 units lattice
run 75000

In [ ]:
!"E:/LAMMPS/bin/lmp" < Fe.in

## 完整的辐照损伤任务
注：
- 体系大小应该随着PKA原子能量的变化而变化，设定的PKA原子能量越大，那么构建的体系也应该越大，要保证级联碰撞过程处于体系中心，不会影响边界原子~
- 速度的大小和方向也可以类似温度一样设置为变量，方便后续更改
- 影响级联碰撞模拟结果的参数有两个：
    - 温度
    - PKA原子能量

In [ ]:
%%writefile Fe_loop.in
package omp 8
#定义循环变量
variable i loop 10
variable seed index 6 8 9 7 2 77 88 66 99 17
variable T equal 300
#系统初始化
units metal 
atom_style atomic
boundary p p p
#初始模型
lattice bcc 2.8552
region box block 0 30 0 30 0 30 units lattice
create_box 1 box
create_atoms 1 box
#选取中心原子为pka原子
region rpka sphere 15 15 15 0.2 units lattice
group  pka region rpka
#定义原子间相互作用势
pair_style eam/fs
pair_coeff * * Fe_mm.eam.fs Fe
#定义输出
thermo 2000
thermo_style custom step temp etotal time
dump 1 all custom 500 Fe.${i}.${T}.xyz id type x y z
#模拟环境设定
velocity all create ${T} ${seed} rot yes dist gaussian
fix 1 all nvt temp ${T} ${T} 0.05
timestep 0.001
run 15000
unfix 1
velocity pka set 300 500 600 units box
fix 1 all nve 
fix 2 all dt/reset 1 0.00000001 0.0005 0.005 units lattice
run 75000
print "************************************"
print "JOB ${i} done"
print "************************************"
#进行循环
clear 
next i 
next seed
jump Fe_loop.in
quit

In [ ]:
!"E:/LAMMPS/bin/lmp" < Fe_loop.in